🤖 **[View in NotebookLM Research Assistant](https://notebooklm.cloud.google.com/us/notebook/e1e83661-33c7-459f-85cd-565bba46d9b6?project=294334118581)**

_This notebook is automatically synced to NotebookLM daily for AI-powered research assistance._

In [1]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Optional, Sequence, Union, Mapping, TypedDict

import html
import pandas as pd
from IPython.display import HTML, display
from pandas.io.formats.style import Styler
from plotly.graph_objs import Figure as PlotlyFigure
import copy

# Public API
__all__ = [
    "display_chart",
    "display_two_charts",
    "display_two_charts_fixed_size",
    "display_table_and_chart",
    "display_table_and_two_charts",
    "display_table_and_two_charts_flexible",
]

# -----------------------------
# Internal utilities
# -----------------------------

# Default chart size
MIN_CHART_WIDTH_PX = 600
MIN_CHART_HEIGHT_PX = 400

Content = Union[pd.DataFrame, Styler, PlotlyFigure, str]


class PlotlyConfig(TypedDict, total=False):
    responsive: bool
    displayModeBar: bool
    displaylogo: bool
    toImageButtonOptions: Mapping[str, object]


@dataclass(frozen=True)
class FlexDefaults:
    container_width: str = "100%"
    container_height: Optional[str] = None
    gap: str = "16px"
    padding: str = "10px"
    border: str = "1px solid #e5e7eb"
    border_radius: str = "6px"
    background: str = "#fff"
    justify: str = "flex-start"
    align: str = "flex-start"
    responsive: bool = True


def _copy_figure(fig: PlotlyFigure) -> PlotlyFigure:
    """Safely copy a Plotly figure without mutating the original."""
    try:
        return copy.deepcopy(fig)
    except Exception:
        return PlotlyFigure(fig.to_dict())


def _df_to_html(df: pd.DataFrame) -> str:
    """Render a DataFrame to simple HTML."""
    return df.to_html(border=0, classes=["dataframe"], escape=False)


def _styler_to_html(styler: Styler) -> str:
    return styler.to_html()


def _figure_to_html(
    fig: PlotlyFigure,
    *,
    chart_width: Optional[int] = None,
    chart_height: Optional[int] = None,
    config: Optional[PlotlyConfig] = None,
) -> str:
    fig_copy = _copy_figure(fig)

    # Use provided dimensions or defaults
    w = chart_width or MIN_CHART_WIDTH_PX
    h = chart_height or MIN_CHART_HEIGHT_PX

    fig_copy.update_layout(
        width=w,
        height=h,
        autosize=True,
        margin=dict(l=40, r=40, t=80, b=40),  # Increased top margin for legend
    )

    cfg: PlotlyConfig = {
        "responsive": True,
        "displayModeBar": True,
        "displaylogo": False,
    }
    if config:
        cfg.update(config)

    return fig_copy.to_html(include_plotlyjs="cdn", full_html=False, config=cfg)


def _content_to_html(
    content: Content,
    name: str = "content",
    chart_width: Optional[int] = None,
    chart_height: Optional[int] = None,
    *,
    plotly_config: Optional[PlotlyConfig] = None,
    trust_raw_html: bool = True,
) -> str:
    """Normalize content to HTML."""
    if isinstance(content, Styler):
        return _styler_to_html(content)
    if isinstance(content, pd.DataFrame):
        return _df_to_html(content)
    if isinstance(content, PlotlyFigure):
        return _figure_to_html(
            content,
            chart_width=chart_width,
            chart_height=chart_height,
            config=plotly_config,
        )
    if isinstance(content, str):
        return content if trust_raw_html else html.escape(content)

    raise TypeError(
        f"`{name}` must be a DataFrame, Styler, Plotly Figure, or HTML string. Got {type(content)!r}"
    )


def _display_flex(
    html_blocks: Sequence[str],
    widths: Sequence[str],
    *,
    container_width: str = FlexDefaults.container_width,
    container_height: Optional[str] = FlexDefaults.container_height,
    padding: str = FlexDefaults.padding,
    responsive: bool = FlexDefaults.responsive,
    justify: str = FlexDefaults.justify,
    align: str = FlexDefaults.align,
    gap: str = FlexDefaults.gap,
    flex_grows: Optional[Sequence[str]] = None,
    border: str = FlexDefaults.border,
    border_radius: str = FlexDefaults.border_radius,
    background: str = FlexDefaults.background,
    role: str = "group",
    overflow_x: Optional[str] = None,
) -> None:
    """Render multiple HTML blocks side by side in a flex container."""
    if len(html_blocks) != len(widths):
        raise ValueError("Length of widths must match number of html blocks.")

    if flex_grows and len(flex_grows) != len(html_blocks):
        raise ValueError("Length of flex_grows must match number of html blocks.")

    # Container style
    styles = [
        "display:flex;",
        f"justify-content:{justify};",
        f"align-items:{align};",
        f"width:{container_width};",
        f"gap:{gap};",
        "box-sizing:border-box;",
    ]

    if overflow_x:
        styles.append(f"overflow-x:{overflow_x};")
    elif not responsive:
        styles.append("overflow-x:auto;")

    if container_height:
        styles.append(f"height:{container_height};")

    styles.append("flex-wrap:wrap;" if responsive else "flex-wrap:nowrap;")

    wrapper_style = "".join(styles)

    # Build item panes
    panes = []
    for i, (block_html, width) in enumerate(zip(html_blocks, widths)):
        grow = flex_grows[i] if flex_grows else "0"
        shrink = "0"  # Don't shrink
        basis = width

        pane_style = (
            f"box-sizing:border-box; padding:{padding}; "
            f"flex:{grow} {shrink} {basis}; "
            f"overflow:hidden; "
            f"border:{border}; border-radius:{border_radius}; background:{background};"
        )
        panes.append(f'<div style="{pane_style}">{block_html}</div>')

    html_out = f'<div role="{role}" style="{wrapper_style}">{"".join(panes)}</div>'
    display(HTML(html_out))


# -----------------------------
# Public display helpers
# -----------------------------


def display_chart(
    chart: Union[PlotlyFigure, str],
    *,
    modify_layout: bool = False,
    chart_width_px: Optional[int] = None,
    chart_height_px: Optional[int] = None,
    plotly_config: Optional[PlotlyConfig] = None,
) -> None:
    """
    Display a single chart centered.
    By default, displays the chart as-is without modifications.
    Set modify_layout=True to apply sizing and legend positioning.
    """
    if isinstance(chart, PlotlyFigure):
        if modify_layout:
            # Apply modifications if requested
            chart_html = _content_to_html(
                chart,
                "chart",
                chart_width=chart_width_px or MIN_CHART_WIDTH_PX,
                chart_height=chart_height_px or MIN_CHART_HEIGHT_PX,
                plotly_config=plotly_config,
            )
        else:
            # Display chart as-is, no modifications
            cfg: PlotlyConfig = {
                "responsive": True,
                "displayModeBar": True,
                "displaylogo": False,
            }
            if plotly_config:
                cfg.update(plotly_config)
            chart_html = chart.to_html(
                include_plotlyjs="cdn", full_html=False, config=cfg
            )
    else:
        # If it's already HTML string, use as-is
        chart_html = chart

    # Simple centered layout
    html_out = f"""
    <div style="
        display:flex;
        justify-content:center;
        align-items:center;
        width:100%;
    ">
        <div>
            {chart_html}
        </div>
    </div>
    """
    display(HTML(html_out))


def display_two_charts(
    chart1: Union[PlotlyFigure, str],
    chart2: Union[PlotlyFigure, str],
    *,
    modify_layout: bool = True,
    chart_width_px: Optional[int] = None,
    chart_height_px: Optional[int] = None,
    plotly_config: Optional[PlotlyConfig] = None,
    responsive: bool = True,
) -> None:
    """
    Display two charts side by side using responsive layout.
    By default uses percentage widths for true responsiveness.
    Set modify_layout=False to display charts without size/legend modifications.
    """
    if responsive:
        # Responsive percentage-based layout (recommended)
        if isinstance(chart1, PlotlyFigure) and isinstance(chart2, PlotlyFigure):
            if modify_layout:
                # Apply legend fixes if requested
                chart1 = _copy_figure(chart1)
                chart2 = _copy_figure(chart2)
                for fig in [chart1, chart2]:
                    fig.update_layout(margin=dict(l=40, r=40, t=80, b=40))

            cfg: PlotlyConfig = {
                "responsive": True,
                "displayModeBar": True,
                "displaylogo": False,
            }
            if plotly_config:
                cfg.update(plotly_config)

            c1_html = chart1.to_html(
                full_html=False, include_plotlyjs="cdn", config=cfg
            )
            c2_html = chart2.to_html(
                full_html=False, include_plotlyjs=False, config=cfg
            )
        else:
            c1_html = chart1 if isinstance(chart1, str) else str(chart1)
            c2_html = chart2 if isinstance(chart2, str) else str(chart2)

        # Using f-string to avoid format() issues with CSS braces
        html_out = f"""
        <style>
            .chart-flex-wrapper {{
                display: flex;
                flex-wrap: wrap;
                gap: 20px;
                justify-content: center;
            }}
            .chart-container {{
                flex: 1 1 45%;
                min-width: 300px;
                box-sizing: border-box;
            }}
        </style>
        <div class="chart-flex-wrapper">
            <div class="chart-container">{c1_html}</div>
            <div class="chart-container">{c2_html}</div>
        </div>
        """
    else:
        # Fixed pixel layout (old behavior)
        width = chart_width_px or MIN_CHART_WIDTH_PX
        height = chart_height_px or MIN_CHART_HEIGHT_PX

        c1_html = _content_to_html(
            chart1,
            "chart1",
            chart_width=width,
            chart_height=height,
            plotly_config=plotly_config,
        )
        c2_html = _content_to_html(
            chart2,
            "chart2",
            chart_width=width,
            chart_height=height,
            plotly_config=plotly_config,
        )

        html_out = f"""
        <div style="
            display:flex;
            flex-direction:row;
            flex-wrap:nowrap;
            gap:20px;
            width:100%;
            overflow-x:auto;
            align-items:flex-start;
            justify-content:center;
        ">
            <div style="flex:0 0 {width}px; max-width:{width}px;">
                {c1_html}
            </div>
            <div style="flex:0 0 {width}px; max-width:{width}px;">
                {c2_html}
            </div>
        </div>
        """

    display(HTML(html_out))


def display_two_charts_fixed_size(
    chart1: Union[PlotlyFigure, str],
    chart2: Union[PlotlyFigure, str],
    *,
    chart_width: int = 400,
    chart_height: int = 400,
    container_width: str = "100%",
    gap: str = "20px",
    plotly_config: Optional[PlotlyConfig] = None,
) -> None:
    """Display two charts with fixed pixel dimensions."""

    def prepare(chart: Union[PlotlyFigure, str], name: str) -> str:
        if isinstance(chart, PlotlyFigure):
            fig = _copy_figure(chart)
            fig.update_layout(
                width=chart_width,
                height=chart_height,
                autosize=False,
                margin=dict(l=40, r=40, t=80, b=40),  # Increased top margin for legend
            )
            cfg: PlotlyConfig = {
                "responsive": False,
                "displayModeBar": True,
                "displaylogo": False,
            }
            if plotly_config:
                cfg.update(plotly_config)
            return fig.to_html(include_plotlyjs="cdn", full_html=False, config=cfg)
        return chart

    c1_html = prepare(chart1, "chart1")
    c2_html = prepare(chart2, "chart2")

    html_out = f"""
    <div style="
        display:flex;
        flex-direction:row;
        flex-wrap:nowrap;
        gap:{gap};
        width:{container_width};
        overflow-x:auto;
        align-items:center;
        justify-content:center;
    ">
        <div style="flex:0 0 {chart_width}px;">{c1_html}</div>
        <div style="flex:0 0 {chart_width}px;">{c2_html}</div>
    </div>
    """
    display(HTML(html_out))


def display_table_and_chart(
    table: Union[pd.DataFrame, Styler, str],
    chart: Union[PlotlyFigure, str],
    *,
    table_width: str = "30%",
    chart_width: str = "70%",
    chart_first: bool = False,
    chart_width_px: Optional[int] = None,
    chart_height_px: Optional[int] = None,
    plotly_config: Optional[PlotlyConfig] = None,
    **flex_kwargs,
) -> None:
    """Display a table and a chart side by side."""
    table_html = _content_to_html(table, "table")
    chart_html = _content_to_html(
        chart, "chart", chart_width_px, chart_height_px, plotly_config=plotly_config
    )

    contents = [table_html, chart_html]
    widths = [table_width, chart_width]

    if chart_first:
        contents.reverse()
        widths.reverse()

    defaults = {
        "container_height": "500px",
        "gap": "15px",
        "responsive": False,
    }
    defaults.update(flex_kwargs)

    _display_flex(contents, widths, **defaults)


def display_table_and_two_charts(
    table: Union[pd.DataFrame, Styler, str],
    chart1: Union[PlotlyFigure, str],
    chart2: Union[PlotlyFigure, str],
    *,
    chart_width_px: Optional[int] = None,
    chart_height_px: Optional[int] = None,
    plotly_config: Optional[PlotlyConfig] = None,
    **flex_kwargs,
) -> None:
    """
    Display a table and two charts side by side.
    Table uses natural width; charts split remaining space equally using flex-grow.
    """
    table_html = _content_to_html(table, "table")
    c1_html = _content_to_html(
        chart1, "chart1", chart_width_px, chart_height_px, plotly_config=plotly_config
    )
    c2_html = _content_to_html(
        chart2, "chart2", chart_width_px, chart_height_px, plotly_config=plotly_config
    )

    contents = [table_html, c1_html, c2_html]
    widths = ["auto", "1px", "1px"]  # minimal flex-basis for charts
    flex_grows = ["0", "1", "1"]  # charts grow equally

    defaults = {
        "container_height": "500px",
        "gap": "15px",
        "responsive": False,
        "justify": "flex-start",
        "flex_grows": flex_grows,
    }
    defaults.update(flex_kwargs)

    _display_flex(contents, widths, **defaults)


def display_table_and_two_charts_flexible(
    table: Union[pd.DataFrame, Styler, str],
    chart1: Union[PlotlyFigure, str],
    chart2: Union[PlotlyFigure, str],
    *,
    table_width: str = "auto",
    gap_width: int = 8,
    chart_width_px: Optional[int] = None,
    chart_height_px: Optional[int] = None,
    plotly_config: Optional[PlotlyConfig] = None,
) -> None:
    """Scrollable layout: table + two charts in a single row with horizontal scroll on overflow."""

    table_html = _content_to_html(table, "table")
    c1_html = _content_to_html(
        chart1, "chart1", chart_width_px, chart_height_px, plotly_config=plotly_config
    )
    c2_html = _content_to_html(
        chart2, "chart2", chart_width_px, chart_height_px, plotly_config=plotly_config
    )

    html_out = f"""
    <div style="
        display:flex;
        flex-wrap:nowrap;
        align-items:flex-start;
        gap:{gap_width}px;
        overflow-x:auto;
        overflow-y:visible;
        -webkit-overflow-scrolling:touch;
        padding:4px 2px;
        width:100%;
        box-sizing:border-box;
    ">
        <div style="flex:0 0 auto; min-width:{table_width};">{table_html}</div>
        <div style="flex:0 0 auto;">{c1_html}</div>
        <div style="flex:0 0 auto;">{c2_html}</div>
    </div>
    """
    display(HTML(html_out))

In [2]:
import pandas as pd
from IPython.display import Markdown, HTML
from tulip.data.bloomberg import BloombergClient as bb
from tulip.data.haver import HaverClient as hc
from tulip.plots import plot_lines
from tulip.core import *
from tulip.data.gs import GSClient as gs
from tulip.analysis.country_related.analytics import summarize_gs_eco_fct

from tulip.core import merge_collections
from tulip.genai import iris

Haver path setting remains unchanged.



In [3]:
# Country
ctry_2 = "EU"
ctry_3 = "EUR"
ccy = "EUR"
ctry_name = "Eurozone"
ccy_2 = "EU"  # bloombergs currency
goldman_ctry = "EAagg"
bloomberg_ctry = "EZ"
haver_num = "025"

## Eurozone
### Activity Indicators
#### Economic Forecasts (Brokers)

In [4]:
gs_eco_fct = gs.get_eco_forecast(geographyId=goldman_ctry)
gs_summary = summarize_gs_eco_fct(gs_eco_fct)
gs_summary = gs_summary[~gs_summary.index.str.contains("ngdp")].to_frame().T
gs_summary.style.set_caption(f"Goldman {ctry_name} Economic Forecasts").format(
    precision=2
)

metric,core_cpi,cpi_avg,current_account,rgdp_qoq,rgdp_yoy,output_gap
forecastValue,2.42,2.13,nan,1.28,1.44,1.44


#### Current Activity Indicator (Goldman)

In [5]:
cai_series_soft_vs_hard = gs.get_CAI_series(
    geographyId=goldman_ctry,
    metricName=[
        "CAI_HEADLINE",
        "CAI_CONTRIBUTION_TYPE_HARD",
        "CAI_CONTRIBUTION_TYPE_SOFT",
    ],
    startDate="1980-01-01",
)
cai_last_month = cai_series_soft_vs_hard.index.max().to_period("M")
cai_update_time = cai_series_soft_vs_hard.updateTime.max().strftime("%Y-%m-%d")
cai_series_soft_vs_hard = gs.get_CAI_series(
    geographyId=goldman_ctry,
    metricName=[
        "CAI_HEADLINE",
        "CAI_CONTRIBUTION_TYPE_HARD",
        "CAI_CONTRIBUTION_TYPE_SOFT",
    ],
    startDate="1980-01-01",
)
cai_series_soft_vs_hard = cai_series_soft_vs_hard.set_index("metricName", append=True)[
    "metricValue"
].unstack("metricName")
cai_series_soft_vs_hard.columns = ["Hard", "Soft", "Headline"]
cai_plot = plot_lines(
    cai_series_soft_vs_hard,
    show_0=True,
    title=f"<b>{ctry_name} Current Activity Indicator</b> Updated: {cai_update_time}",
    years_limit=4,
    source=f"Source: Goldman Sachs (Last data point: {cai_last_month})",
    tick_suffix="%",
)
display_chart(cai_plot)

In [6]:
cai_headline_individual_ctrys = gs.get_CAI_series(
    geographyId=[goldman_ctry, "DE", "FR", "NL", "ES", "PL", "IT"],
    metricName=[
        "CAI_HEADLINE",
    ],
    startDate="1980-01-01",
)
cai_headline_individual_ctrys = cai_headline_individual_ctrys.reset_index().pivot(
    index="date", columns="geographyName", values="metricValue"
)
core_cai_plot = plot_lines(
    cai_headline_individual_ctrys[["Euro Area", "France", "Germany", "Netherlands"]],
    years_limit=3,
    show_0=True,
    title=f"<b>Core CAI</b> Updated: {pd.Timestamp.today().strftime('%Y-%m-%d')}",
    tick_suffix="%",
)
periphery_cai_plot = plot_lines(
    cai_headline_individual_ctrys[["Euro Area", "Spain", "Italy", "Poland"]],
    years_limit=3,
    show_0=True,
    title=f"<b>Periphery Countries CAI</b> Updated: {pd.Timestamp.today().strftime('%Y-%m-%d')}",
    tick_suffix="%",
)
display_two_charts(core_cai_plot, periphery_cai_plot)

In [7]:
nowcasts = bb.create_collection(
    [
        f"BENWEAGC Index",
        f"BENWDEGC Index",
        "BENWFRGC Index",
        "BENWITGC Index",
        "BENWESGC Index",
    ]
)
nowcasts.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Quote Units,Updated
Bloomberg Economics Euro Area,0.57,2025-11-12,0.56,0.01,0.13,0.19,0.47,-,2025-11-12 19:19:00
Bloomberg Economics Germany GD,0.33,2025-11-12,0.32,0.01,0.09,0.12,0.26,-,2025-11-12 19:19:00
Bloomberg Economics France GDP,0.49,2025-11-12,0.47,0.02,0.13,0.17,0.32,-,2025-11-12 19:19:00
Bloomberg Economics Italy GDP,0.51,2025-11-12,0.49,0.02,0.12,0.12,0.18,-,2025-11-12 19:19:00
Bloomberg Economics Spain GDP,0.93,2025-11-12,0.90,0.03,0.14,0.17,0.51,-,2025-11-12 19:19:00


- European Commission [Nowcasts](https://web.jrc.ec.europa.eu/rapps/pub/ea-nowcasting/) and its real time app [Nowcast](https://inflation-nowcast-socialplatform.streamlit.app/)

#### Retail sales
The indexes below are Retail Trade Ex Autos, Motorcycles & Fuel **Volume** index. The closest series to US Retail Sales

In [8]:
retail_sales = {
    "European Union (EU27)": "S997D4YK@EUDATA",
    "Euro Area 20": "S025D4YK@EUDATA",
    "France": "S132D4YK@EUDATA",
    "Germany": "S134D4YK@EUDATA",
    "Italy": "S136D4YK@EUDATA",
    "Spain": "S184D4YK@EUDATA",
    "Netherlands": "S138D4YK@EUDATA",
    "Poland": "S964D4YK@EUDATA",
}

retail_sales_collection = hc.create_collection(retail_sales)
retail_sales_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
European Union (EU27),103.3,2025-09-30,103.2,0.10,0.08,1.3,1.2,2025-11-12 19:19:00
Euro Area 20,102.7,2025-09-30,102.6,0.10,0.07,1.2,0.90,2025-11-12 19:19:00
France,106.8,2025-09-30,106.8,0.00,0.00,2.0,1.9,2025-11-12 19:19:00
Germany,100.0,2025-09-30,99.3,0.70,0.44,0.80,0.20,2025-11-12 19:19:00
Italy,95.2,2025-09-30,95.7,-0.50,-0.26,-0.80,-2.0,2025-11-12 19:19:00
Spain,111.5,2025-09-30,111.0,0.50,0.27,3.4,4.6,2025-11-12 19:19:00
Netherlands,102.0,2025-09-30,102.3,-0.30,-0.20,1.0,1.4,2025-11-12 19:19:00
Poland,113.2,2025-09-30,112.7,0.50,0.32,3.8,3.5,2025-11-12 19:19:00


In [9]:
retail_sales_id_to_title = {
    s.info.id.split("@")[0].lower(): s.info.title
    for s in retail_sales_collection.tulips
}
retail_sales_df = retail_sales_collection.df.rename(columns=retail_sales_id_to_title)

retail_sales_plot = plot_lines(
    retail_sales_df.iloc[:, :2],
    title="<b>Retail Sales EU and Eurozone</b>",
    tick_suffix="%",
    show_0=True,
    years_limit=10,
).add_hline(y=100, line_width=1)

retail_sales_chg_plot = plot_lines(
    retail_sales_df.iloc[:, :2].pct_change(12, fill_method=None),
    title="<b>Retail Sales EU and Eurozone YoY</b>",
    tick_format="0.0%",
    show_0=True,
    years_limit=3,
)

display_two_charts(retail_sales_plot, retail_sales_chg_plot)


In [10]:
retail_sales_core_plot = plot_lines(
    retail_sales_df.loc[
        :,
        ["European Union (EU27)", "Euro Area 20", "France", "Netherlands"],
    ],
    title="<b></b>",
    tick_suffix="%",
    show_0=True,
    years_limit=10,
    default_y_range=[60, 120],
    logo=False,
).add_hline(y=100, line_width=1)
retail_sales_periphery_plot = plot_lines(
    retail_sales_df.loc[
        :,
        [
            "European Union (EU27)",
            "Euro Area 20",
            "Italy",
            "Spain",
            "Poland",
        ],
    ],
    title="<b></b>",
    tick_suffix="%",
    show_0=True,
    years_limit=10,
    default_y_range=[60, 120],
    logo=False,
).add_hline(y=100, line_width=1)

display_two_charts(retail_sales_core_plot, retail_sales_periphery_plot)

#### Unemployment Rate

In [11]:
ue_rate = {
    "UE European Union (EU27)": "S997R@EUDATA",
    "UE Euro Area 20": "S025R@EUDATA",
    "UE France": "S132R@EUDATA",
    "UE Germany": "S134R@EUDATA",
    "UE Italy": "S136R@EUDATA",
    "UE Spain": "S184R@EUDATA",
    "UE Netherlands": "S138R@EUDATA",
    "UE Poland": "S964R@EUDATA",
}

ue_rate_collection = hc.create_collection(ue_rate)
ue_rate_collection.good_is = -1
ue_rate_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
UE European Union (EU27),6.0,2025-09-30,6.0,0.00,0.00,0.10,0.10,2025-11-12 19:19:00
UE Euro Area 20,6.3,2025-09-30,6.3,0.00,0.00,0.00,0.00,2025-11-12 19:19:00
UE France,7.6,2025-09-30,7.5,0.10,0.80,0.10,0.20,2025-11-12 19:19:00
UE Germany,3.9,2025-09-30,3.8,0.10,1.2,0.30,0.50,2025-11-12 19:19:00
UE Italy,6.1,2025-09-30,6.0,0.10,0.46,0.00,0.00,2025-11-12 19:19:00
UE Spain,10.5,2025-09-30,10.5,0.00,0.00,-0.30,-0.60,2025-11-12 19:19:00
UE Netherlands,4.0,2025-09-30,3.9,0.10,0.86,0.20,0.30,2025-11-12 19:19:00
UE Poland,3.2,2025-09-30,3.2,0.00,0.00,0.10,0.20,2025-11-12 19:19:00


In [12]:
cpi_id_to_title = {
    s.info.id.split("@")[0].lower(): s.info.title for s in ue_rate_collection.tulips
}
ue_rate_df = ue_rate_collection.df.rename(columns=cpi_id_to_title)
ue_core_plot = plot_lines(
    ue_rate_df.loc[
        :,
        ["UE European Union (EU27)", "UE Euro Area 20", "UE France", "UE Netherlands"],
    ],
    title="<b></b>",
    tick_suffix="%",
    show_0=True,
    years_limit=10,
    default_y_range=[0, 17],
).add_hline(y=100, line_width=1)
ue_periphery_plot = plot_lines(
    ue_rate_df.loc[
        :,
        [
            "UE European Union (EU27)",
            "UE Euro Area 20",
            "UE Italy",
            "UE Spain",
            "UE Poland",
        ],
    ],
    title="<b></b>",
    tick_suffix="%",
    show_0=True,
    years_limit=10,
    default_y_range=[0, 17],
)
display_two_charts(ue_core_plot, ue_periphery_plot)


#### PMIs

In [13]:
pmi = {
    "MPMIEZCA Index": "Eurozone Composite PMI",
    "MPMIEZMA Index": "Eurozone Manufacturing PMI",
    "MPMIDECA Index": "German Composite PMI",
    "MPMIDEMA Index": "German Manufacturing PMI",
    "MPMIFRCA Index": "France Composite PMI",
    "MPMIFRMA Index": "France Manufacturing PMI",
}

pmi_collection = bb.create_collection(list(pmi.keys()))
for i, k in enumerate(pmi.values()):
    pmi_collection[i].info.title = k

pmi_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Quote Units,Updated
Eurozone Composite PMI,52.5,2025-10-31,51.2,1.3,1.1,2.1,2.5,-,2025-11-12 19:19:00
Eurozone Manufacturing PMI,50.0,2025-10-31,49.8,0.20,0.20,1.0,4.0,-,2025-11-12 19:19:00
German Composite PMI,53.9,2025-10-31,52.0,1.9,1.1,3.8,5.3,-,2025-11-12 19:19:00
German Manufacturing PMI,49.6,2025-10-31,49.5,0.10,0.07,1.2,6.6,-,2025-11-12 19:19:00
France Composite PMI,47.7,2025-10-31,48.1,-0.40,-0.23,-0.10,-0.40,-,2025-11-12 19:19:00
France Manufacturing PMI,48.8,2025-10-31,48.2,0.60,0.38,0.10,4.3,-,2025-11-12 19:19:00


In [14]:
pmi_id_to_title = {s.info.id: s.info.title for s in pmi_collection.tulips}
fig = plot_lines(
    pmi_collection.df.rename(columns=pmi_id_to_title), logo=False
).add_hline(y=50, line_width=1)
display_chart(fig)

#### Industrial Production

In [15]:
indprod = {
    "IP European Union (EU27)": "S997QC@EUDATA",
    "IP Euro Area 20": "S025QC@EUDATA",
    "IP France": "S132QC@EUDATA",
    "IP Germany": "S134QC@EUDATA",
    "IP Italy": "S136QC@EUDATA",
    "IP Spain": "S184QC@EUDATA",
    "IP Netherlands": "S138QC@EUDATA",
    "IP Poland": "S964QC@EUDATA",
}

indprod_collection = hc.create_collection(indprod)
indprod_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
IP European Union (EU27),100.3,2025-08-31,101.3,-1.0,-0.64,-0.20,0.90,2025-11-12 19:19:00
IP Euro Area 20,98.7,2025-08-31,99.9,-1.2,-0.73,-0.60,0.80,2025-11-12 19:19:00
IP France,103.8,2025-09-30,102.8,1.0,0.43,1.5,1.5,2025-11-12 19:19:00
IP Germany,91.6,2025-09-30,89.9,1.7,0.86,-0.80,-0.90,2025-11-12 19:19:00
IP Italy,94.7,2025-09-30,93.4,1.3,0.47,2.0,1.2,2025-11-12 19:19:00
IP Spain,103.8,2025-09-30,102.4,1.4,0.59,2.6,1.3,2025-11-12 19:19:00
IP Netherlands,104.4,2025-09-30,104.3,0.10,0.07,0.80,2.1,2025-11-12 19:19:00
IP Poland,117.6,2025-09-30,112.7,4.9,2.7,7.1,8.8,2025-11-12 19:19:00


In [16]:
ip_id_to_title = {
    s.info.id.split("@")[0].lower(): s.info.title for s in indprod_collection.tulips
}
indprod_df = indprod_collection.df.rename(columns=ip_id_to_title)
ip_plot = plot_lines(
    indprod_df.iloc[:, :2],
    title="<b>Industrial Production EU and Eurozone</b>",
    tick_suffix="%",
    show_0=True,
    years_limit=10,
).add_hline(y=100, line_width=1)
ip_chg_plot = plot_lines(
    indprod_df.iloc[:, :2].pct_change(12, fill_method=None),
    title="<b>Industrial Production EU and Eurozone YoY</b>",
    tick_format="0.0%",
    show_0=True,
    years_limit=3,
)
display_two_charts(ip_plot, ip_chg_plot)


In [17]:
ip_core_plot = plot_lines(
    indprod_df.loc[:, ["IP France", "IP Germany", "IP Netherlands"]],
    title="<b>Industrial Production Core</b>",
    tick_suffix="%",
    show_0=True,
    years_limit=10,
).add_hline(y=100, line_width=1)
ip_periphery_plot = plot_lines(
    indprod_df.loc[:, ["IP Italy", "IP Spain", "IP Poland"]],
    title="<b>Industrial Production Periphery</b>",
    tick_suffix="%",
    show_0=True,
    years_limit=10,
).add_hline(y=100, line_width=1)
display_two_charts(ip_core_plot, ip_periphery_plot)

In [18]:
growth_conditions = merge_collections(
    [retail_sales_collection, ue_rate_collection, pmi_collection, indprod_collection]
)
growth_cond_ai = iris.summarize(
    growth_conditions,
    custom_prompt="While summarizing please highlight potential differences between core "
    "(Germany, France, Netherlands) and periphery (Italy, Spain, Poland) countries."
    " Only if they are relevant. If not just say they are not relevant."
    " Do not presume the core goes better than the periphery.",
)
growth_cond_ai.html()

ECB [Macroeconomic Projections](https://www.ecb.europa.eu/press/projections/html/index.en.html)

#### Growth Stats
##### Real GDP Growth

In [19]:
real_gdp_eu_col = hc.create_collection(
    [
        "F025GDPY@EUDATA",
        "F134GDPY@EUDATA",
        "F132GDPY@EUDATA",
        "F136GDPY@EUDATA",
        "F184GDPY@EUDATA",
    ]
)
real_gdp_df = real_gdp_eu_col.ts
real_gdp_df.columns = ["Eurozone", "Germany", "France", "Italy", "Spain"]
plot_lines(
    real_gdp_df,
    title="Eurozone Real GDP Growth",
    show_0=True,
    source="Haver",
    tick_suffix="%",
    years_limit=5,
)

In [20]:
real_gdp_eu_col.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Germany: Preliminary GDP Flash Estimate in Chained Prices (SWDA, Y/Y % Change)",0.30,2025-09-30,0.40,-0.10,-0.03,0.50,0.50,2025-11-12 19:19:00
"Italy: Preliminary GDP Flash Estimate in Chained Prices(SWDA, Y/Y % Change)",0.40,2025-09-30,0.40,0.00,0.00,-0.10,0.00,2025-11-12 19:19:00
"France: Preliminary GDP Flash Estimate in Chained Prices (SWDA, Y/Y % Change)",0.90,2025-09-30,0.70,0.20,0.03,0.20,-0.40,2025-11-12 19:19:00
"Spain: Preliminary GDP Flash Estimate in Chained Prices(SWDA, Y/Y % Change)",2.8,2025-09-30,2.8,0.00,0.00,-0.70,-0.60,2025-11-12 19:19:00
"EA 19-20: Preliminary GDP Flash Estimate in Chained Prices (SWDA, Y/Y % Change)",1.3,2025-09-30,1.4,-0.10,-0.02,0.40,0.40,2025-11-12 19:19:00


##### GDP Detail

In [21]:
growth_stats = {
    "Real GDP": f"J{haver_num}TCK@EUDATA",
    "Fixed Investment": f"J{haver_num}IFK@EUDATA",
    "Private Consumption": f"J{haver_num}PCK@EUDATA",
    "Govt Consumption": f"J{haver_num}GCK@EUDATA",
    "Exports": f"J{haver_num}EXPK@EUDATA",
    "Imports": f"J{haver_num}IMPK@EUDATA",  # Imports has to go last, so we can flip good_is
}

growth_stats_collection = hc.create_collection(list(growth_stats.values()))
growth_stats_collection[-1].good_is = -1  # Flip what Good is
growth_stats_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"EA20: Final Consumption Exp: Contr to Real GDP Growth(SWDA, %Y/Y)",1.1,2025-06-30,1.2,-0.08,-0.05,-0.17,0.19,2025-11-12 19:19:00
"EA20: Gross Fixed Cap Formation: Contr to Real GDP Growth(SWDA, %Y/Y)",0.63,2025-06-30,0.49,0.14,0.14,1.0,1.3,2025-11-12 19:19:00
"EA20: Private Consumption: Contr to Real GDP Growth(SWDA, %Y/Y)",0.77,2025-06-30,0.77,0.00,0.00,0.02,0.35,2025-11-12 19:20:00
"EA20: Gen Govt Final Cons Exp: Contr to Real GDP Growth(SWDA, %Y/Y)",0.34,2025-06-30,0.42,-0.08,-0.36,-0.19,-0.16,2025-11-12 19:20:00
"EA20: Imports of G&S: Contributions to Real GDP Growth(SWDA, %Y/Y)",-1.2,2025-06-30,-1.8,0.63,0.34,-0.46,-1.4,2025-11-12 19:20:00
"EA20: Exports of G&S: Contributions to Real GDP Growth(SWDA, %Y/Y)",0.21,2025-06-30,1.3,-1.1,-0.57,-0.31,-0.55,2025-11-12 19:20:00


In [22]:
gdp_question = iris.summarize(
    growth_stats_collection,
    custom_prompt="Analyze what is driving Eurozone GDP growth.Remember Imports are a drag on growth and that the eurozone requires exports.",
)
gdp_question.html()

In [23]:
growth_stats_collection.dashboard.plots(show_0=True, years_limit=10)

##### Nominal GDP

In [24]:
#  Nominal GDP Growth
nom_gdp = hc.get_series(f"J{haver_num}GDPE@EUDATA")
nom_gdp_fig = hc.get_series(f"yryr%(J{haver_num}GDPE@EUDATA)").plot(
    show_0=True,
    tick_suffix="%",
)  # Nominal GDP Growth
display_chart(nom_gdp_fig)

### Prices

In [25]:
prices = {
    "EA11-20 HICP As Reported": "yryr%(F023H@EUDATA)",  # Eurozone
    "EA11-20 HICP": "yryr%(P023H@EUDATA)",
    "EA11-20 HICP Core": "yryr%(P023HOEF@EUDATA)",
    "EA20 HICP ": "yryr%(H025H@EUDATA)",
    "EA20 HICP Core": " yryr%(H025HOEF@EUDATA)",  # Disc
    "EA20 HICP Goods": "yryr%(H025HG@EUDATA)",
    "EA20 HICP Services": "yryr%(H025HS@EUDATA)",
}

prices_collection = hc.create_collection(prices)
prices_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
EA11-20 HICP As Reported,2.1,2025-10-31,2.2,-0.10,-0.24,-0.05,0.11,2025-11-12 19:20:00
EA11-20 HICP,2.1,2025-10-31,2.2,-0.13,-0.44,-0.06,0.10,2025-11-12 19:20:00
EA11-20 HICP Core,2.4,2025-10-31,2.4,0.01,0.06,-0.38,-0.33,2025-11-12 19:20:00
EA20 HICP,2.2,2025-09-30,2.1,0.18,0.55,-0.09,0.49,2025-11-12 19:20:00
EA20 HICP Core,2.4,2025-09-30,2.3,0.10,0.61,-0.20,-0.29,2025-11-12 19:20:00
EA20 HICP Goods,1.4,2025-09-30,1.1,0.22,0.41,0.15,1.4,2025-11-12 19:20:00
EA20 HICP Services,3.2,2025-09-30,3.1,0.17,0.88,-0.46,-0.70,2025-11-12 19:20:00


In [26]:
prices_df = prices_collection.df
prices_df.columns = list(prices.keys())
fig = plot_lines(
    prices_df.iloc[:, -4:],
    years_limit=8,
    title="<b>Key Inflation Measures</b>",
    tick_suffix="%",
    source="ECB/Eurostat",
)
display_chart(
    fig.add_hline(2, line_width=1, annotation_text="Target")
)  # , annotation="Target")

In [27]:
prices_per_country = {
    "EA20 HICP ": "yryr%(H025H@EUDATA)",
    "EA20 HICP Core": " yryr%(H025HOEF@EUDATA)",
    "German HICP ": "yryr%(H134H@EUDATA)",
    "German HICP Core": " yryr%(H134HOEF@EUDATA)",
    "France HICP ": "yryr%(H132H@EUDATA)",
    "France HICP Core": " yryr%(H132HOEF@EUDATA)",
    "Italy HICP ": "yryr%(H136H@EUDATA)",
    "Italy HICP Core": " yryr%(H136HOEF@EUDATA)",
    "Spain HICP ": "yryr%(H184H@EUDATA)",
    "Spain HICP Core": " yryr%(H184HOEF@EUDATA)",
    "Netherlands HICP ": "yryr%(H138H@EUDATA)",
    "Netherlands HICP Core": " yryr%(H138HOEF@EUDATA)",
    "Poland HICP ": "yryr%(H964H@EUDATA)",
    "Poland HICP Core": " yryr%(H964HOEF@EUDATA)",
}

prices_per_country_collection = hc.create_collection(prices_per_country)
prices_per_country_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
EA20 HICP,2.2,2025-09-30,2.1,0.18,0.55,-0.09,0.49,2025-11-12 19:20:00
EA20 HICP Core,2.4,2025-09-30,2.3,0.10,0.61,-0.20,-0.29,2025-11-12 19:20:00
German HICP,2.3,2025-10-31,2.4,-0.09,-0.19,0.37,-0.05,2025-11-12 19:20:00
German HICP Core,2.5,2025-09-30,2.4,0.16,0.47,-0.55,-0.50,2025-11-12 19:20:00
France HICP,0.86,2025-10-31,1.1,-0.21,-0.70,-0.03,-0.72,2025-11-12 19:20:00
France HICP Core,1.5,2025-10-31,1.6,-0.08,-0.38,-0.38,-0.61,2025-11-12 19:20:00
Italy HICP,1.3,2025-10-31,1.8,-0.49,-1.2,-0.66,0.32,2025-11-12 19:20:00
Italy HICP Core,1.9,2025-10-31,2.1,-0.18,-0.57,-0.19,0.05,2025-11-12 19:20:00
Spain HICP,3.2,2025-10-31,3.0,0.16,0.35,1.1,1.3,2025-11-12 19:20:00
Spain HICP Core,2.7,2025-09-30,2.6,0.09,0.41,0.45,-0.01,2025-11-12 19:20:00


In [28]:
prices_per_country_df = prices_per_country_collection.df
prices_per_country_df.columns = list(prices_per_country.keys())

In [29]:
overall_prices_fig = plot_lines(
    prices_per_country_df.iloc[:, :2],
    years_limit=3,
    tick_suffix="%",
    title="<b>Eurozone Inflation</b>",
    logo=False,
).add_hline(2, line_width=1)
for tr in overall_prices_fig.data:
    if "Core" in (tr.name or ""):
        tr.update(line=dict(dash="dot"))
display_chart(overall_prices_fig)

In [30]:
# todo: add titles to the plots
core_cpi = prices_per_country_df.loc[
    :,
    [
        "EA20 HICP ",
        "EA20 HICP Core",
        "German HICP ",
        "German HICP Core",
        "France HICP ",
        "France HICP Core",
        "Netherlands HICP ",
        "Netherlands HICP Core",
    ],
]
periphery_cpi = prices_per_country_df.loc[
    :,
    [
        "EA20 HICP ",
        "EA20 HICP Core",
        "Spain HICP ",
        "Spain HICP Core",
        "Italy HICP ",
        "Italy HICP Core",
        "Poland HICP ",
        "Poland HICP Core",
    ],
]

core_fig = plot_lines(
    core_cpi,
    years_limit=3,
    tick_suffix="%",
    title="",
    logo=False,
).add_hline(2, line_width=1)
for tr in core_fig.data:
    if "Core" in (tr.name or ""):
        tr.update(line=dict(dash="dot"))
    if "EA20" in (tr.name or ""):
        tr.update(line=dict(color="blue"))
    if "France" in (tr.name or ""):
        tr.update(line=dict(color="red"))
    if "German" in (tr.name or ""):
        tr.update(line=dict(color="black"))
periphery_fig = plot_lines(
    periphery_cpi,
    years_limit=3,
    tick_suffix="%",
    title="",
    logo=False,
).add_hline(2, line_width=1)
for tr in periphery_fig.data:
    if "Core" in (tr.name or ""):
        tr.update(line=dict(dash="dot"))
    if "EA20" in (tr.name or ""):
        tr.update(line=dict(color="blue"))
    if "Italy" in (tr.name or ""):
        tr.update(line=dict(color="green"))
    if "Spain" in (tr.name or ""):
        tr.update(line=dict(color="brown"))
    if "Poland" in (tr.name or ""):
        tr.update(line=dict(color="lightblue"))
display_two_charts(core_fig, periphery_fig)

In [31]:
inflation_expectations = {
    "Three Quarters Ahead": "V025IM23@EUDATA",
    "Seven Quarters Ahead": "V025IM27@EUDATA",
    "Current Year": "V025IM2C@EUDATA",
    "Next year": "V025IM2N@EUDATA",
    "Two years Ahead": "V025IM2Y@EUDATA",
    "Long term": "V025IM2L@EUDATA",
}

inflation_expectations_collection = hc.create_collection(
    list(inflation_expectations.values())
)
inflation_expectations_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
EA 11-20: SPF Inflation Forecast: Mean Pt Est [3-digit]: Long-Term (Y/Y % Chg),2.0,2025-12-31,2.0,0.01,0.30,-0.01,0.02,2025-11-12 19:20:00
EA 11-20: SPF Inflation Forecast: Mean Pt Est [3-digit]: 3 Qtrs Ahead(Y/Y % Chg),1.8,2025-12-31,1.8,-0.02,-0.09,-0.18,-0.16,2025-11-12 19:20:00
EA 11-20: SPF Inflation Forecast: Mean Pt Est [3-digit]: Next Year (Y/Y % chg),1.8,2025-12-31,1.8,-0.01,-0.01,-0.15,-0.12,2025-11-12 19:20:00
EA 11-20: SPF Inflation Forecast: Mean Pt Est [3-digit]: 2 Yrs Ahead (Y/Y % Chg),2.0,2025-12-31,2.0,-0.00,-0.03,-0.03,0.07,2025-11-12 19:20:00
EA 11-20: SPF Inflation Forecast: Mean Pt Est [3-digit]: Current Year(Y/Y % chg),2.1,2025-12-31,2.0,0.05,0.08,-0.09,-0.28,2025-11-12 19:20:00
EA 11-20: SPF Inflation Forecast: Mean Pt Est [3-digit]: 7 Qtrs Ahead(Y/Y % Chg),1.9,2025-12-31,2.0,-0.04,-0.48,-0.09,0.04,2025-11-12 19:20:00


In [32]:
inflation_expectations_df = inflation_expectations_collection.df
inflation_expectations_df.columns = inflation_expectations.keys()
display(HTML("<b>ECB Survey of Professional Forecasters</b>"))
fig = plot_lines(inflation_expectations_df, tick_suffix="%", years_limit=5, logo=False)
fig.add_hline(2, line_width=1, annotation_text="Target")  # ,
# Color traces from dark (short horizon) to light (long term)
palette = ["#08306B", "#08519C", "#2171B5", "#4292C6", "#6BAED6", "#C6DBEF"]
for i, trc in enumerate(fig.data):
    if i < len(palette):
        trc.update(line=dict(color=palette[i]))

display_chart(fig)

In [33]:
prices_col = merge_collections(
    [prices_per_country_collection, inflation_expectations_collection]
)
response = iris.summarize(
    prices_col,
    custom_prompt="While summarizing please highlight potential differences between core "
    "(Germany, France, Netherlands) and periphery (Italy, Spain, Poland) countries."
    " Only if they are relevant. If not just say they are not relevant."
    " Do not presume the core goes better than the periphery.",
)
response.html()

In [34]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")

_Notebook updated at 2025-11-12 19:21_